In [4]:
import requests
import pandas as pd

# Replace with your actual API key from CryptoRank
API_KEY = "2b6fe8ab443c10bb3f559991e807f359797f0a3b8a3a6c5dd1f1cba4237f"

# Define the API endpoint and parameters
url = "https://api.cryptorank.io/v2/currencies/map"
params = {
    "include": ["lifeCycle", "rank", "type"]  # optional includes
}
headers = {
    "X-Api-Key": API_KEY
}

# Make the request
response = requests.get(url, params=params, headers=headers)

# Check for success
if response.status_code == 200:
    data = response.json()["data"]
    df = pd.DataFrame(data)
    display(df.head())  # Display the first few rows
else:
    print("Error fetching data:", response.status_code, response.text)


,id,key,symbol,name,lifeCycle,rank,type
0,1,bitcoin,BTC,Bitcoin,traded,1.0,coin
1,2,eos,EOS,EOS,traded,70.0,coin
2,3,ethereum,ETH,Ethereum,traded,2.0,coin
3,4,neo,NEO,NEO,traded,150.0,coin
4,5,ripple,XRP,XRP,traded,4.0,coin


In [5]:
df

,id,key,symbol,name,lifeCycle,rank,type
0,1,bitcoin,BTC,Bitcoin,traded,1.0,coin
1,2,eos,EOS,EOS,traded,70.0,coin
2,3,ethereum,ETH,Ethereum,traded,2.0,coin
3,4,neo,NEO,NEO,traded,150.0,coin
4,5,ripple,XRP,XRP,traded,4.0,coin
...,...,...,...,...,...,...,...
33036,198738,peep-the-toad,PEEP,Peep The Toad,traded,1750.0,token
33037,198739,gorecats,GCATS,GORECATS,traded,NaN,token
33038,198740,disney-plus,Disney++,Disney++,traded,1862.0,token
33039,198741,first-ever-meme-currency,Kekels,First Ever Meme Currency,traded,1798.0,token


In [6]:
# Filter the dataframe where the column 'type' is 'coin'
coins_df = df[df['type'] == 'coin']

# Save the filtered dataframe to a CSV file
coins_df.to_csv('coinsList.csv', index=False)

In [1]:
import requests
import pandas as pd
from datetime import datetime

def safe_get(data, keys, default=None):
    """Safely navigate nested dictionaries"""
    for key in keys:
        try:
            data = data.get(key, default)
        except AttributeError:
            return default
        if data is None:
            return default
    return data

def fetch_crypto_data(timestamp: str):
    """Fetch data with enhanced error handling"""
    try:
        url = f"https://cryptorank.io/_next/data/{timestamp}/en/all-coins-list.json"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json().get('pageProps', {}).get('coins', [])
    except Exception as e:
        print(f"API Error: {str(e)}")
        return []

def parse_coin_data(coins):
    """Parse data with complete null safety"""
    datasets = {
        'base': [],
        'prices': [],
        'supply': [],
        'history': [],
        'volatility': [],
        'tags': [],
        'categories': []
    }

    for coin in coins:
        # Base Metadata
        datasets['base'].append({
            'key': safe_get(coin, ['key']),
            'name': safe_get(coin, ['name']),
            'symbol': safe_get(coin, ['symbol']),
            'rank': safe_get(coin, ['rank'], 0),
            'type': safe_get(coin, ['type'], 'unknown'),
            'life_cycle': safe_get(coin, ['lifeCycle'], 'unknown'),
            'full_name': safe_get(coin, ['fullName']),
            'image_url': safe_get(coin, ['image']),
            'listing_date': safe_get(coin, ['listingDate']),
            'has_funding': safe_get(coin, ['hasFundingRounds'], False),
            'market_data': not safe_get(coin, ['marketDataNotAvailable'], True),
            'interest': safe_get(coin, ['interest'], 0)
        })

        # Category Data
        category = safe_get(coin, ['category'], {})
        datasets['categories'].append({
            'key': safe_get(coin, ['key']),
            'category': safe_get(category, ['name']),
            'category_key': safe_get(category, ['key'])
        })

        # Supply Metrics
        datasets['supply'].append({
            'key': safe_get(coin, ['key']),
            'max_supply': safe_get(coin, ['maxSupply'], 0),
            'total_supply': safe_get(coin, ['totalSupply'], 0),
            'available_supply': safe_get(coin, ['availableSupply'], 0),
            'circulating_pct': safe_get(coin, ['percentOfCircSupply'], 0),
            'fdv_mcap': safe_get(coin, ['fullyDilutedMarketCap'], 0)
        })

        # Price Data
        price_data = safe_get(coin, ['price'], {})
        datasets['prices'].append({
            'key': safe_get(coin, ['key']),
            'price_usd': safe_get(coin, ['priceUsd'], 0),
            'price_btc': safe_get(price_data, ['BTC'], 0),
            'price_eth': safe_get(price_data, ['ETH'], 0),
            'market_cap': safe_get(coin, ['marketCap'], 0),
            'volume_24h': safe_get(coin, ['volume24hUsd'], 0)
        })

        # Historical Data
        ath_price = safe_get(coin, ['athPrice'], {})
        ath_mcap = safe_get(coin, ['athMarketCap'], {})
        atl_price = safe_get(coin, ['atlPrice'], {})
        datasets['history'].append({
            'key': safe_get(coin, ['key']),
            'ath_usd': safe_get(ath_price, ['USD'], 0),
            'ath_btc': safe_get(ath_price, ['BTC'], 0),
            'ath_eth': safe_get(ath_price, ['ETH'], 0),
            'ath_date': safe_get(ath_price, ['date']),
            'ath_mcap': safe_get(ath_mcap, ['USD'], 0),
            'ath_mcap_date': safe_get(ath_mcap, ['dateUSD']),
            'atl_usd': safe_get(atl_price, ['USD'], 0),
            'atl_date': safe_get(atl_price, ['dateUSD'])
        })

        # Volatility Data
        volatility = safe_get(coin, ['volatility'], {})
        if any(volatility.values()):
            datasets['volatility'].append({
                'key': safe_get(coin, ['key']),
                'usd_vol': safe_get(volatility, ['USD'], 0),
                'eth_vol': safe_get(volatility, ['ETH'], 0)
            })

        # Tag Relationships
        for tag_id in safe_get(coin, ['tagIds'], []):
            datasets['tags'].append({
                'key': safe_get(coin, ['key']),
                'tag_id': tag_id
            })

    # Convert to DataFrames
    return {k: pd.DataFrame(v) for k, v in datasets.items()}

def clean_data(df_dict):
    """Apply final data cleaning"""
    for name, df in df_dict.items():
        # Convert dates
        date_cols = [c for c in df.columns if 'date' in c]
        for col in date_cols:
            df[col] = pd.to_datetime(df[col], errors='coerce')
        
        # Convert numerics
        num_cols = [c for c in df.columns if df[c].dtype == object]
        num_cols = [c for c in num_cols if c not in date_cols]
        df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='ignore')
    
    return df_dict

# Usage Example
timestamp = "1744894442079"  # Replace with actual timestamp
raw_data = fetch_crypto_data(timestamp)
if raw_data:
    datasets = parse_coin_data(raw_data)
    cleaned = clean_data(datasets)
    
    print("Base Metadata:")
    print(cleaned['base'].head(3))
else:
    print("No data received")

Base Metadata:
        key      name symbol  rank   type life_cycle       full_name  \
0   bitcoin   Bitcoin    BTC     1   coin     traded   Bitcoin (BTC)   
1  ethereum  Ethereum    ETH     2   coin     traded  Ethereum (ETH)   
2    tether    Tether   USDT     3  token     traded   Tether (USDT)   

                                           image_url  \
0  https://img.cryptorank.io/coins/60x60.bitcoin1...   
1  https://img.cryptorank.io/coins/60x60.ethereum...   
2  https://img.cryptorank.io/coins/60x60.tether16...   

               listing_date  has_funding  market_data  interest  
0 2010-03-17 00:00:00+00:00        False         True         0  
1 2015-07-30 00:00:00+00:00        False         True         0  
2 2015-02-09 00:00:00+00:00        False         True         0  


C:\Users\mahmo\AppData\Local\Temp\ipykernel_27008\822528547.py:131: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='ignore')
C:\Users\mahmo\AppData\Local\Temp\ipykernel_27008\822528547.py:131: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='ignore')
C:\Users\mahmo\AppData\Local\Temp\ipykernel_27008\822528547.py:131: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='ignore')
C:\Users\mahmo\AppData\Local\Temp\ipykernel_27008\822528547.py:131: FutureWarning: errors='ignore' is depreca

In [2]:
if raw_data:
    datasets = parse_coin_data(raw_data)
    cleaned = clean_data(datasets)
    
    # Display all datasets with formatting
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', 4)
    pd.set_option('display.width', 1000)
    
    print("="*80 + "\nComplete Dataset Overview\n" + "="*80)
    for dataset_name, df in cleaned.items():
        print(f"\n{'='*40}\n{dataset_name.upper()} DATASET\n{'='*40}")
        print(f"Shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
        print("\nSample Data:")
        print(df.head(3) if not df.empty else print("No data"))
        print("\n" + "-"*80)
    
    # Reset display options
    pd.reset_option('display.max_columns')
    pd.reset_option('display.max_rows')
    pd.reset_option('display.width')
else:
    print("No data received")


Complete Dataset Overview

BASE DATASET
Shape: (100, 12)
Columns: ['key', 'name', 'symbol', 'rank', 'type', 'life_cycle', 'full_name', 'image_url', 'listing_date', 'has_funding', 'market_data', 'interest']

Sample Data:
        key      name symbol  rank   type life_cycle       full_name                                          image_url              listing_date  has_funding  market_data  interest
0   bitcoin   Bitcoin    BTC     1   coin     traded   Bitcoin (BTC)  https://img.cryptorank.io/coins/60x60.bitcoin1... 2010-03-17 00:00:00+00:00        False         True         0
1  ethereum  Ethereum    ETH     2   coin     traded  Ethereum (ETH)  https://img.cryptorank.io/coins/60x60.ethereum... 2015-07-30 00:00:00+00:00        False         True         0
2    tether    Tether   USDT     3  token     traded   Tether (USDT)  https://img.cryptorank.io/coins/60x60.tether16... 2015-02-09 00:00:00+00:00        False         True         0

---------------------------------------------------

C:\Users\mahmo\AppData\Local\Temp\ipykernel_27008\822528547.py:131: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='ignore')
C:\Users\mahmo\AppData\Local\Temp\ipykernel_27008\822528547.py:131: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='ignore')
C:\Users\mahmo\AppData\Local\Temp\ipykernel_27008\822528547.py:131: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='ignore')
C:\Users\mahmo\AppData\Local\Temp\ipykernel_27008\822528547.py:131: FutureWarning: errors='ignore' is depreca

In [10]:
import time
import requests
import pandas as pd
from datetime import datetime
import os
from tqdm import tqdm

BASE_URL = "https://cryptorank.io/_next/data/1744894442079/en/price/{key}/analytics.json"

def safe_get(data, path, default=None):
    if isinstance(path, str):
        path = path.split('/')
    current = data
    for key in path:
        if isinstance(current, list) and key.isdigit():
            key = int(key)
        try:
            current = current[key]
        except (KeyError, TypeError, IndexError):
            return default
    return current

def fetch_coin_data(coin_key):
    try:
        response = requests.get(
            BASE_URL.format(key=coin_key),
            headers={'User-Agent': 'Mozilla/5.0'},
            timeout=15
        )
        response.raise_for_status()
        return response.json().get('pageProps', {})
    except Exception as e:
        return None

def parse_coin_data(data):
    if not data:
        return None

    coin_info = {
        'name': safe_get(data, ['name']),
        'symbol': safe_get(data, ['symbol']),
        'rank': safe_get(data, ['rank'], 0),
        'price_usd': safe_get(data, ['priceUsd'], 0),
        'market_cap': safe_get(data, ['marketCap'], 0),
        'volume_24h': safe_get(data, ['volume24hUsd'], 0),
        'circulating_supply': safe_get(data, ['availableSupply'], 0),
        'max_supply': safe_get(data, ['maxSupply'], 0),
        'all_time_high': safe_get(data, ['athPrice', 'USD'], 0),
        'ath_date': safe_get(data, ['athPrice', 'date']),
        'category': safe_get(data, ['category']),
        'launch_date': safe_get(data, ['listingDate']),
        'website': next((link['value'] for link in safe_get(data, ['links'], []) 
                         if link['type'] == 'web'), None)
    }

    historical_prices = []
    hist_prices = safe_get(data, ['histPrices'], {})
    for period, values in hist_prices.items():
        historical_prices.append({
            'period': period,
            'price_usd': safe_get(values, ['USD'], 0),
            'price_btc': safe_get(values, ['BTC'], 0),
            'price_eth': safe_get(values, ['ETH'], 0)
        })

    team_members = []
    for member in safe_get(data, ['team'], []):
        team_members.append({
            'name': safe_get(member, ['name']),
            'position': safe_get(member, ['jobs', 0]) if safe_get(member, ['jobs']) else 'N/A',
            'links': ', '.join([link['value'] for link in safe_get(member, ['links'], [])])
        })

    crowdsales = []
    for sale in safe_get(data, ['crowdsales'], []):
        crowdsales.append({
            'type': safe_get(sale, ['type']),
            'date': safe_get(sale, ['end']),
            'price': safe_get(sale, ['price', 'USD'], 0),
            'raised': safe_get(sale, ['raise', 'USD'], 0)
        })

    allocations = []
    for alloc in safe_get(data, ['icoData', 'allocationChart'], []):
        allocations.append({
            'category': safe_get(alloc, ['title']),
            'percentage': safe_get(alloc, ['percent'], 0)
        })

    return {
        'coin_info': coin_info,
        'historical_prices': historical_prices,
        'team': team_members,
        'crowdsales': crowdsales,
        'allocations': allocations
    }

if __name__ == "__main__":
    # Replace this with your actual source of coin keys
    coins_df_copy = coins_df.head(1000)
    coin_keys = coins_df_copy['key']


    os.makedirs('data', exist_ok=True)
    errors = []

    coin_info_all = []
    historical_prices_all = []
    team_all = []
    crowdsales_all = []
    allocations_all = []

    for coin_key in tqdm(coin_keys, desc="Fetching coin data"):
        raw_data = fetch_coin_data(coin_key)
        time.sleep(0.25)  # 4 requests per second

        if raw_data and (coin_data := raw_data.get('coin')):
            parsed = parse_coin_data(coin_data)
            if parsed:
                parsed['coin_info']['coin_key'] = coin_key
                coin_info_all.append(parsed['coin_info'])

                for item in parsed['historical_prices']:
                    item['coin_key'] = coin_key
                    historical_prices_all.append(item)

                for item in parsed['team']:
                    item['coin_key'] = coin_key
                    team_all.append(item)

                for item in parsed['crowdsales']:
                    item['coin_key'] = coin_key
                    crowdsales_all.append(item)

                for item in parsed['allocations']:
                    item['coin_key'] = coin_key
                    allocations_all.append(item)
            else:
                print(f"⚠️ Parse failed: {coin_key}")
                errors.append(f"{coin_key} (parse failed)")
        else:
            print(f"❌ Fetch failed: {coin_key}")
            errors.append(f"{coin_key} (fetch failed)")

    # Save final combined CSVs
    pd.DataFrame(coin_info_all).to_csv('data/coin_info_e.csv', index=False)
    pd.DataFrame(historical_prices_all).to_csv('data/historical_prices_e.csv', index=False)
    pd.DataFrame(team_all).to_csv('data/team_e.csv', index=False)
    pd.DataFrame(crowdsales_all).to_csv('data/crowdsales_e.csv', index=False)
    pd.DataFrame(allocations_all).to_csv('data/allocations_e.csv', index=False)

    # Save error logs
    with open('data/errors.log', 'w') as f:
        for line in errors:
            f.write(line + '\n')

    print("\n✅ Data collection complete. Files saved to `data/` folder.")


Fetching coin data:   6%|▌         | 58/1000 [00:42<11:35,  1.35it/s]

❌ Fetch failed: wax


Fetching coin data:   6%|▌         | 60/1000 [00:43<09:50,  1.59it/s]

❌ Fetch failed: ubiq


Fetching coin data:   6%|▌         | 61/1000 [00:44<09:36,  1.63it/s]

...

❌ Fetch failed: conceal


Fetching coin data: 100%|█████████▉| 999/1000 [13:41<00:00,  1.71it/s]

❌ Fetch failed: wine-shares


Fetching coin data: 100%|██████████| 1000/1000 [13:42<00:00,  1.22it/s]

❌ Fetch failed: coral-dex

✅ Data collection complete. Files saved to `data/` folder.
